In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
        

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Conv2DTranspose, BatchNormalization, UpSampling2D, Reshape, Dropout
from keras import backend as K
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
df_train =  pd.read_csv('data/train_labels.csv')
df_train

,filename,width,height,class,xmin,ymin,xmax,ymax
0,00000000a9764df2_20191005_085747_4.png,524,750,cappy_portakal,60,7,210,90
1,00000000a9764df2_20191005_085747_4.png,524,750,fuse_tea_seftali,217,1,396,78
2,00000000a9764df2_20191005_085747_4.png,524,750,johnnie_walker,390,1,439,155
3,00000000a9764df2_20191005_085747_4.png,524,750,miller,65,86,387,199
4,00000000a9764df2_20191005_085747_4.png,524,750,kinder_joy,438,83,503,193
...,...,...,...,...,...,...,...,...
11075,00000000a9764df2_20191003_163704_4.png,524,750,campari,383,26,452,176
11076,00000000a9764df2_20191003_163704_4.png,524,750,baileys,68,189,215,251
11077,00000000a9764df2_20191003_163704_4.png,524,750,kahlua,97,519,223,567
11078,00000000a9764df2_20191003_163704_4.png,524,750,beefeater,409,534,471,664


In [4]:
df_test =  pd.read_csv('data/test_labels.csv')
df_test

,filename,width,height,class,xmin,ymin,xmax,ymax
0,00000000a9764df2_20191005_030806_4.png,524,750,nescafe_xpress,57,7,225,88
1,00000000a9764df2_20191005_030806_4.png,524,750,burn,231,1,424,74
2,00000000a9764df2_20191005_030806_4.png,524,750,sprite,65,116,250,196
3,00000000a9764df2_20191005_030806_4.png,524,750,fanta,280,75,463,188
4,00000000a9764df2_20191005_030806_4.png,524,750,tadim_kavrulmus_badem,65,501,206,686
...,...,...,...,...,...,...,...,...
3995,00000000a9764df2_20191003_190301_4.png,524,750,efes_malt,60,320,452,441
3996,00000000a9764df2_20191003_190301_4.png,524,750,redbull_sugar_free,81,501,260,565
3997,00000000a9764df2_20191003_190301_4.png,524,750,efes_extra_shot,344,500,468,552
3998,00000000a9764df2_20191003_190301_4.png,524,750,pepsi,64,590,249,683


In [5]:
def decouple(df):
    matrix = {}
    classes = {}
    for index, row in df.iterrows():
        if row['filename'] not in matrix:
            matrix[row['filename']] = {'count':0}
        if row['class'] not in matrix[row['filename']]:
            matrix[row['filename']][row['class']] = 1
        else:
            matrix[row['filename']][row['class']] += 1
        matrix[row['filename']]['count'] += 1

        if row['class'] not in classes:
            classes[row['class']] = 1
        else:
            classes[row['class']] += 1
    #classes = {k: v for k, v in sorted(classes.items(), key=lambda item: item[1])}            
    return matrix,classes

In [6]:
matrix_train,classes_train = decouple(df_train)
matrix_test,classes_test = decouple(df_test)

In [8]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,validation_split=0.2)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory='data/train',
        x_col='filename',
        y_col='class',
        target_size=(440, 440),
        batch_size=8,
        class_mode='categorical',
        subset="training",)

validation_generator = train_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory='data/train',
        x_col='filename',
        y_col='class',
        target_size=(440, 440),
        batch_size=8,
        class_mode='categorical',
        subset="validation",)



Found 8864 validated image filenames belonging to 40 classes.
Found 2216 validated image filenames belonging to 40 classes.


In [9]:
model = Sequential()

# Step 1 - Convolution
model.add(Conv2D(32, (3, 3), padding='same', input_shape = (440, 440, 3), activation = 'relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5)) # antes era 0.25
# Adding a second convolutional layer
model.add(Conv2D(64, (3, 3), padding='same', activation = 'relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5)) # antes era 0.25
# Adding a third convolutional layer
model.add(Conv2D(64, (3, 3), padding='same', activation = 'relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5)) # antes era 0.25
# Step 3 - Flattening
model.add(Flatten())
# Step 4 - Full connection
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dropout(0.5)) 
model.add(Dense(units = 40, activation = 'softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.save("simple_cnn_classification.h5")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 440, 440, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 438, 438, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 219, 219, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 219, 219, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 219, 219, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 217, 217, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 108, 108, 64)     

In [10]:
model.layers[0].input_shape

(None, 440, 440, 3)

In [11]:
history = model.fit_generator(
        train_generator,
        #steps_per_epoch=100,
        epochs=1,
        #steps_per_epoch=100,
        validation_data=validation_generator,
        #validation_steps=100
        )
model.save("simple_cnn_classification.h5")

Epoch 1/1
 236/1108 [=====>........................] - ETA: 1:26:56 - loss: 3.6707 - accuracy: 0.0461

KeyboardInterrupt: 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()